<a href="https://colab.research.google.com/github/cristiandarioortegayubro/Moodle/blob/master/Encuesta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('Encuesta.txt', 'https://github.com/cristiandarioortegayubro/Moodle/blob/master/Encuesta.txt')

   8192/Unknown - 0s 0us/step

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 240778 characters


In [5]:
print(text[:250])







<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="h


In [6]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

106 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  '\r':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\n\n\n\n\n<!DOCTY' ---- characters mapped to int ---- > [ 1  1  1  1  1  1 29  4 37 48 36 53 58]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch"'
' href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars0.githubuser'
'content.com">\n  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">\n  <link rel="'
'dns-prefetch" href="https://avatars2.githubusercontent.com">\n  <link rel="dns-prefetch" href="https:/'
'/avatars3.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch'
Target data: '\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch"'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    1
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    2
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    3
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    4
  input: 1 ('\n')
  expected output: 1 ('\n')


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 106) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           27136     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 106)           108650    
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([  8,  17,  57,  48,  71,  15,  85,  36,  64,  68,   8,  71,   6,
        29, 105, 102,  28,  91,   8,  33,  95,  28,   3,  91,   5,  66,
        83, 100,  47,   8,  17,  21,  95,   4,  63,  61,  45,  43,   2,
         0,  20,  47,  73,  96,  99,   6,  28, 102,  11,  37,  88,  19,
        31,  11,  78,  99,  34,   5,  38,   7,  64,  97,  33,  72, 103,
         0,  85,  25,  54,  43,  25,  18,  92,   5,  29,  65, 100,  42,
        91,  54,  85,  19,  17,  66,   1,  49,  44, 100,  80,  88,  85,
        11,  90,  75,  97,  32,  33,  84,  44,  98])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '"1"></td>\n        <td id="LC1" class="blob-code blob-code-inner js-file-line">ENCUESTA DE INCIDENTES'

Next Char Predictions: 
 '&0XOj.xCcg&j#<↵ó;·&@Í; ·"evíN&04Í!b`LJ\r\t3NlÑé#;ó)D{2>)qéA"E%cÚ@kú\tx8UJ81¿"<díI·Ux20e\nPKís{x)°nÚ?@wKá'


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 106)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.6635118


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
37/37 [==============================] - 2s 54ms/step - loss: 3.8935
Epoch 2/10
37/37 [==============================] - 2s 52ms/step - loss: 2.5650
Epoch 3/10
37/37 [==============================] - 2s 52ms/step - loss: 1.8292
Epoch 4/10
37/37 [==============================] - 2s 52ms/step - loss: 1.4611
Epoch 5/10
37/37 [==============================] - 2s 52ms/step - loss: 1.2631
Epoch 6/10
37/37 [==============================] - 2s 53ms/step - loss: 1.1388
Epoch 7/10
37/37 [==============================] - 2s 53ms/step - loss: 1.0386
Epoch 8/10
37/37 [==============================] - 2s 53ms/step - loss: 0.9503
Epoch 9/10
37/37 [==============================] - 2s 54ms/step - loss: 0.8693
Epoch 10/10
37/37 [==============================] - 2s 53ms/step - loss: 0.8017


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            27136     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 106)            108650    
Total params: 4,074,090
Trainable params: 4,074,090
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"RESPUESTAS "))

</td>
      </tr>
      <tr>
        <td id="L68" class="blob-num js-line-number" data-line-number="36"></td>
       <td id="L63" class="blob-num js-line-number" data-line-number="301"></td>
</td>
      </tr>
      <tr>
</td>
      </tr>
      <tr>
       <td id="L240" class="blob-num js-line-number" data-cine-number="503"></td>
</td>
      </tr>
    id="L412" class="blob-num js-line-number" data-line-number" data-line-number="204"></td>
        <td id="LC529" 


In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [37]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.662336349487305
Epoch 1 Loss 2.9775
Time taken for 1 epoch 3.233853340148926 sec

Epoch 2 Batch 0 Loss 2.9274654388427734
Epoch 2 Loss 2.1120
Time taken for 1 epoch 2.0463290214538574 sec

Epoch 3 Batch 0 Loss 2.0432708263397217
Epoch 3 Loss 1.7399
Time taken for 1 epoch 2.046013116836548 sec

Epoch 4 Batch 0 Loss 1.3982681035995483
Epoch 4 Loss 1.2447
Time taken for 1 epoch 2.0631589889526367 sec

Epoch 5 Batch 0 Loss 1.3187611103057861
Epoch 5 Loss 1.3256
Time taken for 1 epoch 2.0916378498077393 sec

Epoch 6 Batch 0 Loss 1.1925995349884033
Epoch 6 Loss 1.0149
Time taken for 1 epoch 2.0815439224243164 sec

Epoch 7 Batch 0 Loss 0.9020484685897827
Epoch 7 Loss 0.9475
Time taken for 1 epoch 2.0868258476257324 sec

Epoch 8 Batch 0 Loss 0.8899851441383362
Epoch 8 Loss 0.9093
Time taken for 1 epoch 2.0864126682281494 sec

Epoch 9 Batch 0 Loss 0.7943480014801025
Epoch 9 Loss 0.8178
Time taken for 1 epoch 2.0856878757476807 sec

Epoch 10 Batch 0 Loss 0.7405799031257629